# The diet problem

[Tutorial](https://nbviewer.jupyter.org/github/Pyomo/PyomoGallery/blob/master/diet/DietProblem.ipynb)

In [1]:
import random
import re

from pyomo import environ as pyo
from pyomo.dataportal import DataPortal
from pyomo.opt import SolverFactory

random.seed(123)
infinity = float("inf")

## Problem

Satisfy the daily nutrient requirements at a minimum cost.

### Sets

- $F$: set of foods
- $N$: set of nutrients

### Parameters

- $c_i$: cost per serving of food $i$, $\forall i \in F$
- $a_{ij}$: cost of nutrient $j$ in food $i$, $\forall i \in F$, $\forall j \in N$
- $Nmin_j$, $Nmax_j$: minmum and maximum level of nutrient $j$, $\forall j \in N$
- $V_i$: volume per serving of food $j$, $\forall j \in F$
- $Vmax$: maximum volume of food consumed

### Variables

- $x_i$: number of servings of food $i$ to consume

### Objective

Minimize the cost of food: $\sum _{i \in F} c_i * x_i$

### Constraints

- limit the nutrient consumption for each nutriend $j \in N$: $Nmin_j \leq \sum_{i \in F} a_{ij} x_i \leq Nmax_j$, $\forall j \in N$
- limit the volume of food consumed: $\sum_{i \in F} V_i x_i \leq Vmax$
- lower bound on food consumption: $x_i \geq 0$, $\forall_i \in F$

## Coding the problem

In [2]:
model = pyo.AbstractModel()

### Sets

In [3]:
model.F = pyo.Set()  # Foods
model.N = pyo.Set()  # Nutrients

### Parameters

In [4]:
model.c = pyo.Param(model.F, within=pyo.PositiveReals)
model.a = pyo.Param(model.F, model.N, within=pyo.NonNegativeReals)
model.Nmin = pyo.Param(model.N, within=pyo.NonNegativeReals, default=0.0)
model.Nmax = pyo.Param(model.N, within=pyo.NonNegativeReals, default=infinity)
model.V = pyo.Param(model.F, within=pyo.PositiveReals)
model.Vmax = pyo.Param(within=pyo.PositiveReals)

### Variables

In [5]:
model.x = pyo.Var(model.F, within=pyo.NonNegativeIntegers)

### Objective

In [6]:
def cost_rule(model):
    return sum(model.c[i] * model.x[i] for i in model.F)


model.cost = pyo.Objective(rule=cost_rule)

### Constraints

In [7]:
def nutrient_rule(model, j):
    value = sum(model.a[i, j] * model.x[i] for i in model.F)
    return model.Nmin[j] <= value <= model.Nmax[j]


def volume_rule(model):
    return sum(model.V[i] * model.x[i] for i in model.F) <= model.Vmax


model.nutrient_limit = pyo.Constraint(model.N, rule=nutrient_rule)
model.volume = pyo.Constraint(rule=volume_rule)

## Data

In [8]:
data = DataPortal()
data.load(filename="data/diet.dat", model=model)
for k, v in data.items():
    print(f"{k}: {v}")
    print("=" * 30)

F: ['Cheeseburger', 'Ham Sandwich', 'Hamburger', 'Fish Sandwich', 'Chicken Sandwich', 'Fries', 'Sausage Biscuit', 'Lowfat Milk', 'Orange Juice']
c: {'Cheeseburger': 1.84, 'Ham Sandwich': 2.19, 'Hamburger': 1.84, 'Fish Sandwich': 1.44, 'Chicken Sandwich': 2.29, 'Fries': 0.77, 'Sausage Biscuit': 1.29, 'Lowfat Milk': 0.6, 'Orange Juice': 0.72}
V: {'Cheeseburger': 4.0, 'Ham Sandwich': 7.5, 'Hamburger': 3.5, 'Fish Sandwich': 5.0, 'Chicken Sandwich': 7.3, 'Fries': 2.6, 'Sausage Biscuit': 4.1, 'Lowfat Milk': 8.0, 'Orange Juice': 12.0}
Vmax: 75.0
N: ['Cal', 'Carbo', 'Protein', 'VitA', 'VitC', 'Calc', 'Iron']
Nmin: {'Cal': 2000, 'Carbo': 350, 'Protein': 55, 'VitA': 100, 'VitC': 100, 'Calc': 100, 'Iron': 100}
Nmax: {'Carbo': 375}
a: {('Cheeseburger', 'Cal'): 510, ('Ham Sandwich', 'Cal'): 370, ('Hamburger', 'Cal'): 500, ('Fish Sandwich', 'Cal'): 370, ('Chicken Sandwich', 'Cal'): 400, ('Fries', 'Cal'): 220, ('Sausage Biscuit', 'Cal'): 345, ('Lowfat Milk', 'Cal'): 110, ('Orange Juice', 'Cal'): 80, 

## Solving

In [9]:
instance = model.create_instance(data=data, name="The Diet Problem")
opt = SolverFactory("glpk")
solution = opt.solve(instance)
print(solution)

    function to express ranged inequality expressions. (called from <ipython-
    input-7-3383a4e93487>:3)
    function to express ranged inequality expressions. (called from <ipython-
    input-7-3383a4e93487>:3)
    function to express ranged inequality expressions. (called from <ipython-
    input-7-3383a4e93487>:3)
    function to express ranged inequality expressions. (called from <ipython-
    input-7-3383a4e93487>:3)
    function to express ranged inequality expressions. (called from <ipython-
    input-7-3383a4e93487>:3)
    function to express ranged inequality expressions. (called from <ipython-
    input-7-3383a4e93487>:3)
    function to express ranged inequality expressions. (called from <ipython-
    input-7-3383a4e93487>:3)

Problem: 
- Name: unknown
  Lower bound: 15.05
  Upper bound: 15.05
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 10
  Number of nonzeros: 77
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  St

In [10]:
for v in instance.component_data_objects(pyo.Objective, active=True):
    print(f"{v.name}: ${pyo.value(v)}")

cost: $15.05


In [11]:
for v in instance.component_data_objects(pyo.Var, active=True):
    if v.value > 0:
        name = re.sub(r"[x\[|\]]", "", v.name)
        print(f"{name}: {v.value}")

Cheeseburger: 4.0
Fish Sandwich: 1.0
Fries: 5.0
Lowfat Milk: 4.0


---

In [12]:
%load_ext watermark
%watermark -d -u -v -iv -b -h -m

Last updated: 2020-12-21

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 20.1.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

Hostname: JHCookMac.local

Git branch: master

pyomo: 5.7.2
re   : 2.2.1

